In [2]:
import os
import csv
from dateutil import parser

def povezi_csv_i_tsv(csv_putanja, tsv_putanja, rezultat_putanja):
    # Učitavanje CSV fajla
    with open(csv_putanja, mode='r', newline='', encoding='utf-8') as csv_file:
        csv_reader = csv.reader(csv_file)
        zaglavlje_csv = next(csv_reader)

        # Pronalaženje indeksa kolone 'Sekvenca'
        sekvenca_indeks = zaglavlje_csv.index('Sekvenca')

        # Kreiranje rečnika za mapiranje sekvenca na redove CSV fajla
        mapa_sekvenci = {red[sekvenca_indeks]: red for red in csv_reader}

    # Učitavanje TSV fajla i povezivanje sa CSV podacima
    with open(tsv_putanja, mode='r', newline='', encoding='utf-8') as tsv_file, \
            open(rezultat_putanja, mode='w', newline='', encoding='utf-8') as rezultat_file:

        tsv_reader = csv.reader(tsv_file, delimiter='\t')
        csv_writer = csv.writer(rezultat_file)

        zaglavlje_tsv = next(tsv_reader)

        # Pronalaženje indeksa kolone 'Accession ID' u TSV fajlu
        accession_id_indeks_tsv = zaglavlje_tsv.index('Accession ID')

        # Pisanje zaglavlja u rezultujući CSV fajl
        csv_writer.writerow(['Accession ID'] + zaglavlje_tsv[1:] + zaglavlje_csv)

        # Povezivanje redova iz CSV i TSV fajlova
        for red_tsv in tsv_reader:
            accession_id_tsv = red_tsv[accession_id_indeks_tsv]

            # Provera da li postoji podudaranje Accession ID-a u CSV fajlu
            if accession_id_tsv in mapa_sekvenci:
                red_csv = mapa_sekvenci[accession_id_tsv]

                # Pisanje povezanog reda u rezultujući CSV fajl
                csv_writer.writerow([accession_id_tsv] + red_tsv[1:] + red_csv[1:])  # Preskakanje prvog elementa jer smo već dodali ID

# Direktorijum "Balkan"
balkan_direktorijum = "C:/Users/Mila/Desktop/Balkan"

# Iteriranje kroz poddirektorijume
for drzava_direktorijum in os.listdir(balkan_direktorijum):
    puna_putanja = os.path.join(balkan_direktorijum, drzava_direktorijum)
    if os.path.isdir(puna_putanja):
        for csv_fajl in os.listdir(puna_putanja):
            if csv_fajl.startswith("povezivanje") and csv_fajl.endswith("N.csv"):
                csv_fajl_putanja = os.path.join(puna_putanja, csv_fajl)

                # Pronalaženje TSV fajla u istom direktorijumu
                tsv_fajlovi = [f for f in os.listdir(puna_putanja) if f.lower().endswith('.tsv')]
                if len(tsv_fajlovi) == 1:
                    tsv_fajl_putanja = os.path.join(puna_putanja, tsv_fajlovi[0])

                    rezultat_putanja = os.path.join(puna_putanja, f"povezivanje_sa_tsv_{os.path.basename(csv_fajl)}")

                    # Pozivanje funkcije za povezivanje CSV i TSV fajlova
                    povezi_csv_i_tsv(csv_fajl_putanja, tsv_fajl_putanja, rezultat_putanja)